In [2]:
import boto3
from botocore.exceptions import ClientError
from typing import List, Dict
import json

In [5]:
class BedrockChat:
    def __init__(self, region: str = "us-east-1"):
        # Initialize Bedrock Runtime client
        self.session = boto3.Session()
        self.client = self.session.client("bedrock-runtime", region_name=region)
        # Model ID for Claude 3 Haiku (you can change this to other models)
        self.model_id = "anthropic.claude-3-haiku-20240307-v1:0"
        # Store conversation history
        self.conversation_history: List[Dict] = []
        
    def chat(self, user_message: str) -> str:
        """
        Send a message to the model and get a response
        """
        # Add user message to conversation history
        self.conversation_history.append({
            "role": "user",
            "content": [{"text": user_message}]
        })
        
        try:
            # Send the message to the model with conversation history
            response = self.client.converse(
                modelId=self.model_id,
                messages=self.conversation_history,
                inferenceConfig={
                    "maxTokens": 512,
                    "temperature": 0.7,
                    "topP": 0.9,
                }
            )
            
            # Extract the response text
            response_text = response["output"]["message"]["content"][0]["text"]
            
            # Add assistant's response to conversation history
            self.conversation_history.append({
                "role": "assistant",
                "content": [{"text": response_text}]
            })
            
            return response_text
            
        except ClientError as e:
            error_msg = f"Error calling Bedrock: {str(e)}"
            print(error_msg)
            return error_msg
    
    def get_conversation_history(self) -> List[Dict]:
        """
        Return the current conversation history
        """
        return self.conversation_history
    
    def clear_conversation(self):
        """
        Clear the conversation history
        """
        self.conversation_history = []

In [6]:
chat = BedrockChat()
    
    # Example conversation
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        break
        
    response = chat.chat(user_input)
    print(f"Assistant: {response}")

UnknownServiceError: Unknown service: 'bedrock-runtime'. Valid service names are: accessanalyzer, account, acm, acm-pca, alexaforbusiness, amp, amplify, amplifybackend, amplifyuibuilder, apigateway, apigatewaymanagementapi, apigatewayv2, appconfig, appconfigdata, appflow, appintegrations, application-autoscaling, application-insights, applicationcostprofiler, appmesh, apprunner, appstream, appsync, athena, auditmanager, autoscaling, autoscaling-plans, backup, backup-gateway, backupstorage, batch, billingconductor, braket, budgets, ce, chime, chime-sdk-identity, chime-sdk-media-pipelines, chime-sdk-meetings, chime-sdk-messaging, cloud9, cloudcontrol, clouddirectory, cloudformation, cloudfront, cloudhsm, cloudhsmv2, cloudsearch, cloudsearchdomain, cloudtrail, cloudwatch, codeartifact, codebuild, codecommit, codedeploy, codeguru-reviewer, codeguruprofiler, codepipeline, codestar, codestar-connections, codestar-notifications, cognito-identity, cognito-idp, cognito-sync, comprehend, compreh